<a href="https://colab.research.google.com/github/Franosei/llm_meets_sports/blob/main/codes/agentic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output
import nest_asyncio
nest_asyncio.apply()

In [2]:
import gc
import tensorflow as tf

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [3]:
!pip install ipykernel llama-index nest_asyncio llama-index-embeddings-huggingface llama-index-embeddings-instructor llama-index-llms-huggingface
clear_output()

In [4]:
from llama_index.core import SimpleDirectoryReader

In [5]:
documents = SimpleDirectoryReader(input_files=['/content/final_doc.pdf']).load_data()

In [6]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=500, chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(documents)

In [7]:
node_metadata = nodes[0].get_content(metadata_mode=True)
print(node_metadata)

page_label: 1
file_name: final_doc.pdf
file_path: /content/final_doc.pdf
file_type: application/pdf
file_size: 28772791
creation_date: 2024-06-10
last_modified_date: 2024-06-10

information
On 1994-01-19 00:00:00, Spain played against Portugal in Vigo, Spain. The match was a Friendly and was played on neutral
ground. The home team Spain scored 2 goals, while the away team Portugal scored 2 goals. 
On 1994-02-09 00:00:00, Spain played against Poland in Santa Cruz de Tenerife, Spain. The match was a Friendly and was
played on neutral ground. The home team Spain scored 1 goals, while the away team Poland scored 1 goals. 
On 1994-02-16 00:00:00, Italy played against France in Naples, Italy. The match was a Friendly and was played on neutral
ground. The home team Italy scored 0 goals, while the away team France scored 1 goals. 
On 1994-02-23 00:00:00, Turkey played against Czech Republic in Istanbul, Turkey. The match was a Friendly and was played
on neutral ground. The home team Turkey sco

In [8]:
import os
from typing import List, Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
HF_TOKEN: Optional[str] = os.getenv("HUGGING_FACE_TOKEN")

In [10]:
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [11]:
Settings.llm = HuggingFaceLLM(model_name="mistralai/Mistral-7B-v0.1")
Settings.embed_model = HuggingFaceEmbedding(model_name="allenai/longformer-base-4096")
clear_output()

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True
)

vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the documnet."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for analysis the the probability of each team wining and the expected number of goals."
    ),
)

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [ ]:
try:
    response = query_engine.query("What is Francis doing")
    print(str(response))
except Exception as e:
    print("An error occurred:", str(e))
